### Анонимные функции

In [ ]:
def foo(x):
    return -x

In [ ]:
sorted([1, 2, 3], key=lambda x: foo(x))

In [ ]:
import pandas as pd

data = pd.DataFrame(['Hello', ',', 'World', '!'], columns=['Hello_World!'])
data['Hello_World!'].apply(lambda x: len(x))

In [ ]:
list(map(lambda x : x ** 2, range(10)))

# Пространства имен [Namespaces]

Пространство имён -- мэппинг из имен переменных в объекты.

В коде может быть несколько пространств имён.


### Локальная область видимости

In [ ]:
a = 1

def foo():
    a = 2
    print(locals())
    print(a)
    
foo()

In [ ]:
def show_scope(x):
    y = " World!"
    print(locals())

show_scope("Hello")

In [ ]:
globals() == locals()

### Глобальная область видимости

In [ ]:
def show_scope(x):
    y = " World!"
    print(globals() == locals())

show_scope("Hello")

### Встроенная область видимости

In [ ]:
globals()['a']

In [ ]:
'range' in globals()

In [ ]:
__builtins__

### Функции создают своё пространство имён

In [ ]:
out_func = 3

def f():
    for i in range(10):
        pass
    in_func = 2

f()

In [ ]:
globals()['out_func']

### Циклы и условия не создают своё пространство имён

In [ ]:
for i in range(3):
    in_for = i ** 2
    pass

print(in_for)

In [ ]:
if True:
    in_if = 2
    
print(in_if)

### *Выражения-генераторы создают

In [ ]:
i = 'Hello'

a = [i ** 2 for i in range(10)]
i

## Правило LEGB

### Local
### Enclosing*
### Global
### Built-in

In [ ]:
global_var = 'global_var'

def test(): 
    local_var = 'local_var'
    print('func:', global_var)  # global_var is in enclosing namespaces
    print('func:', local_var)

test()

print(global_var)
print(local_var)


In [ ]:
global_var = 'global_var'

def test():
    global_var = 'global_var_modified'
    print('func  :', global_var)  # global_var shadows another variable with same name


test()
print('global:', global_var)

In [ ]:
global_var = ['globar_var']

def test():
    global_var.append('global_var_modified')
    print(global_var)

test()
print(global_var)

### global

In [1]:
n_counter = 0

In [4]:
global_var = 'global_var'

def test():
    global n_counter
    n_counter += 1
    global_var = 'global_var_modified'
    print(global_var)  # global_var is in enclosing namespaces


test()
print(global_var)


global_var_modified
global_var


In [5]:
n_counter

3

### Вложенные функции

In [6]:
def outer():
    outer_var = 'foo'
    
    def inner():
        inner_var = 'bar'
        print('inner:', outer_var)
        print('inner:', inner_var)

    inner()
    
    print('outer:', outer_var)
    print('outer:', inner_var)

outer()

inner: foo
inner: bar
outer: foo


NameError: name 'inner_var' is not defined

### Замечение

__Функции имеют доступ к внешним пространствам имён относительно того места где они были _определены_, а не _вызваны_ __

In [47]:
it = -1

def f():
    print(it)


def q(func):
    for it in range(10):
        func()
    print(it)

q(f)

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
9


### Проблема

In [20]:
var

NameError: name 'var' is not defined

In [26]:
def outer():
    global var 
    var = 'var1'

    def inner():
        global var
        var = 'v1_up'
        print('inner :', var)

    inner()    
    print('outer :', var)
    
outer()
print('global:', var)

inner : v1_up
outer : v1_up
global: v1_up


### Решение : nonlocal

In [33]:
def outer():
    var = 'v1'

    def inner():
        nonlocal var
        var = 'v1_up'
        print('inner :', var)

    inner()    
    print('outer :', var)
    
outer()
print('global:', var)

inner : v1_up
outer : v1_up
global: v1_up


### Другое странное решение

In [34]:
def outer(): 
    outer.var = 'v1'

    def inner():
        outer.var = 'v2'
        print('inner:', outer.var)

    inner()
    
    print('outer:', outer.var)

outer()

print(outer.var)

inner: v2
outer: v2
v2


### Где теперь лежит переменная?

In [51]:
def outer():
    nonlocal_var = 'v1'

    def inner():
        print('inner:', nonlocal_var)

    return inner
    
f_inner = outer()

del outer

f_inner()

inner: v1


# Замыкания [Closures]
*In computer programming languages, a closure is a function together with a referencing environment of that function. A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined.*

Существования замыканий следует из правила LEGB и возможностью оперировать с функциями как обьектами.

In [52]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

In [57]:
add_ten = make_adder(10)
add_ten(5)

15

In [58]:
add_two  = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

24

In [59]:
make_adder(2)(7)

9

In [66]:
def cell(value = 10):
    def get():
        return value

    def set(new_value):
        nonlocal value
        value = new_value
        return value
    
    value = 20
    return get, set

get, set = cell()

_ = set(20000000000000)

get()

20000000000000

In [72]:
def make_adder(x):
    
    def add(y):
        return x + y
    
    def update(new_x):
        nonlocal x
        x = new_x

    add.update = update
    
    return add

adder = make_adder(10)
                
adder.update(100)

print(adder(15))

115


In [80]:
adder.__closure__ == adder.update.__closure__

True

# Декораторы

Замыкания как способ быстро изменить поведение функции

In [86]:
import sys

def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return inner

pprint = deprecate(print)

pprint('Hello, World!')

Hello, World!


print is deprecated


### Синтаксис декораторов

In [90]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated # add = deprecated(add)
def add(x, y):
    return x + y

@deprecated
def print_(x):
    print(x)

print_('Hello World!')

Hello World!


print_ is deprecated


### Bananize

In [106]:
from IPython import display

def bananize(func):
    return display.HTML('<img src="http://www.sherv.net/cm/emo/funny/2/big-dancing-banana-smiley-emoticon.gif">')

@bananize
def show(x):
    print(x)
    
show

### Проблема

In [95]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

wrapper
None


### Решение 1

In [96]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Решение 2

In [97]:
import functools

def deprecated(func):
    
    @functools.wraps(func) 
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


<img src='http://i0.kym-cdn.com/photos/images/original/000/384/176/d2f.jpg' />

### Практика :: Декоратор Once

In [105]:
def once(func):
    called = False
    
    def wrapper(*args, **kwargs):
        nonlocal called
        if not called:
            called = True
            return func(*args, **kwargs)

    return wrapper

def f():
    print('Hi!')

f()
f()
print(f())

Hi!
None


### Цепочки декораторов

In [110]:
@deprecated
def f(x):
    return x

f(1)
f(1)
f()

wrapper is deprecated!
wrapper is deprecated!
wrapper is deprecated!
